#RecSys_EDA

<a id="0"></a> <br>
 ## Notebook Plan  
1. [Libraries import](#libraries_import)     
1. [Function Declaration](#functions)
1. [Data Loading and Transformation](#data)
1. [Feature Engineering](#feature)     
1. [EDA](#EDA)         
    1. [Basic EDA (Distributions checking)](#EDA1)
      1. [Columns distribution](#distr)
      2. [Session-related insights](#ses)     
    1. [Deep EDA (Hypotheses checking)](#EDA2)     




## Importing necesssary libraries <a class="anchor" id="libraries_import"></a>

In [ ]:
!pip install pyspark py4j

import os
import sys
import math as m
import time
from tqdm.notebook import tqdm
from google.colab import drive
import pandas as pd
import numpy as np
import datetime
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import matplotlib.pyplot as plt

#pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder,StringIndexer
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import monotonically_increasing_id, pandas_udf,regexp_replace,col
from pyspark.sql.types import IntegerType, FloatType, BooleanType, DateType, StringType
from pyspark.sql.window import Window

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0400553bc7095ff61447a813b9b6dce26699d04810d8e857fa464be47a4c18aa
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Declaring functions <a class="anchor" id="functions"></a>

In [ ]:
def OneHotEncoding(dataframe: pd.DataFrame, input_col: pd.Series) -> pd.DataFrame:

  indexer = StringIndexer(inputCol=input_col, outputCol='output_1')
  indexer_fitted = indexer.fit(dataframe)
  df_indexed = indexer_fitted.transform(dataframe)

  encoder = OneHotEncoder(inputCols=['output_1'], outputCols=['output_2'], dropLast=False)
  df_onehot = encoder.fit(df_indexed).transform(df_indexed)

  df_col_onehot = df_onehot.select('*', vector_to_array('output_2').alias('output_3'))

  num_categories = len(df_col_onehot.first()['output_3'])
  cols_expanded = [(F.col('output_3')[i].alias(f'{indexer_fitted.labels[i]}')) for i in range(num_categories)]
  df_cols_onehot = df_col_onehot.select('*', *cols_expanded)
  df_cols_onehot = df_cols_onehot.select([column for column in df_cols_onehot.columns if column not in ['output_1','output_2','output_3']])
  return df_cols_onehot

def delete_brackets(dataframe: pd.DataFrame, column: pd.Series) -> pd.Series:
    df = dataframe.withColumn(column, F.translate(column, '[]', ' '))
    return df

def change_datatype_of_ohe(dataframe: pd.DataFrame) -> pd.DataFrame:
  for col,col_type in dataframe.dtypes:
    if col_type == 'double':
      dataframe = dataframe.withColumn(col, F.col(col).cast(BooleanType()))
  return dataframe

## Loading & transforming DataFrame <a class="anchor" id="data"></a>

Mounting a csv-file

In [ ]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/data_v2_new.zip'
!unzip '/content/drive/MyDrive/Colab Notebooks/data_v2_old.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/data_v2_new.zip
  inflating: ab_data_new.csv         
  inflating: __MACOSX/._ab_data_new.csv  
Archive:  /content/drive/MyDrive/Colab Notebooks/data_v2_old.zip
  inflating: ab_data_old.csv         
  inflating: __MACOSX/._ab_data_old.csv  


Creating a PySpark session and getting a DataFrame

In [ ]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# os.environ["ARROW_PRE_0_15_IPC_FORMAT"] = "1"

spark = SparkSession.builder\
        .appName('RecSys_EDA')\
        .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)



path_old = '/content/ab_data_old.csv'
path_new = '/content/ab_data_new.csv'

df_old = spark.read.csv(path_old, header=True)
df_new = spark.read.csv(path_new, header=True)
df = df_old.unionByName(df_new, allowMissingColumns=True)
df_spl = df
print('Rows: ', df.count())
df.limit(10).show()

Rows:  105497977
+--------+--------------------+--------------+--------------------+--------------+-------------+--------+--------------------+--------------------+--------------------+
|platform|      utc_event_time|utc_event_date|             user_id|    event_type|ecom.price100|ecom.qty|             ecom.nm|       main_category|        sub_category|
+--------+--------------------+--------------+--------------------+--------------+-------------+--------+--------------------+--------------------+--------------------+
|    Site|2023-07-31 20:52:...|    2023-07-31|61896930866132383...|ec.add_to_cart|      [27700]|     [1]|[2081100339580357...|[6968191755455670...|[1664831343325037...|
|    Site|2023-07-31 20:54:...|    2023-07-31|61896930866132383...|ec.add_to_cart|      [20100]|     [1]|[2150579222891727...|[6968191755455670...|[1664831343325037...|
|    Site|2023-07-31 20:58:...|    2023-07-31|61896930866132383...|ec.add_to_cart|      [27700]|     [1]|[2081100339580357...|[69681917554

Changing column types & names

In [ ]:
start_time = time.time()
print("\x1b[31m\"DataFrame before\"\x1b[0m")
df.printSchema()

#changing names
new_names = ['platform', 'utc_event_time','utc_event_date','user_id','event_type','price','quantity','product_id','main_category','sub_category']
df = df.toDF(*new_names)

#deleting '[' and ']' and '.' from str data
for col in ['user_id',"price","quantity","product_id",'main_category','sub_category']:
  df = delete_brackets(df, col)
df = df.withColumn('event_type', F.regexp_replace('event_type', 'ec.', ''))

print("\x1b[31m\"DataFrame after\"\x1b[0m")
df.printSchema()
print("--- %s seconds ---" % (time.time() - start_time))

"DataFrame before"
root
 |-- platform: string (nullable = true)
 |-- utc_event_time: string (nullable = true)
 |-- utc_event_date: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- ecom.price100: string (nullable = true)
 |-- ecom.qty: string (nullable = true)
 |-- ecom.nm: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)

"DataFrame after"
root
 |-- platform: string (nullable = true)
 |-- utc_event_time: string (nullable = true)
 |-- utc_event_date: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)

--- 0.28206706047058105 seconds ---


In [ ]:
df.limit(10).show()

+--------+--------------------+--------------+--------------------+-----------+------+--------+--------------------+--------------------+--------------------+
|platform|      utc_event_time|utc_event_date|             user_id| event_type| price|quantity|          product_id|       main_category|        sub_category|
+--------+--------------------+--------------+--------------------+-----------+------+--------+--------------------+--------------------+--------------------+
|    Site|2023-07-31 20:52:...|    2023-07-31|61896930866132383...|add_to_cart| 27700|       1| 2081100339580357...| 6968191755455670...| 1664831343325037...|
|    Site|2023-07-31 20:54:...|    2023-07-31|61896930866132383...|add_to_cart| 20100|       1| 2150579222891727...| 6968191755455670...| 1664831343325037...|
|    Site|2023-07-31 20:58:...|    2023-07-31|61896930866132383...|add_to_cart| 27700|       1| 2081100339580357...| 6968191755455670...| 1664831343325037...|
|    Site|2023-07-31 21:00:...|    2023-07-31|

Counting Null/NaN values

In [ ]:
start_time = time.time()
df1 = df.select('utc_event_time','user_id','price','quantity', 'product_id', 'main_category','sub_category')
df2 = df.select([c for c in df.columns if c not in ['utc_event_time','user_id','price','quantity', 'product_id']])

df1.select([(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in df1.columns]).show()
df2.select([(F.count(F.when(F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in df2.columns]).show()
print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+-------+-----+--------+----------+-------------------+-------------------+
|     utc_event_time|user_id|price|quantity|product_id|      main_category|       sub_category|
+-------------------+-------+-----+--------+----------+-------------------+-------------------+
|0.03874054381156522|    0.0|  0.0|     0.0|       0.0|0.03874054381156522|0.03874054381156522|
+-------------------+-------+-----+--------+----------+-------------------+-------------------+

+--------+--------------+----------+-------------------+-------------------+
|platform|utc_event_date|event_type|      main_category|       sub_category|
+--------+--------------+----------+-------------------+-------------------+
|     0.0|           0.0|       0.0|0.03874054381156522|0.03874054381156522|
+--------+--------------+----------+-------------------+-------------------+

--- 3171.9269003868103 seconds ---


In [ ]:
#Drop missing values
df = df.dropna(how='any')

Checking 'quantity' for reasonable values

In [ ]:
#df.select('quantity').distinct().collect()

In [ ]:
# replace 0 with 1
df = df.withColumn("quantity", F.when(F.col("quantity") == 0, 1).otherwise(F.col("quantity")))
# df.select('quantity').distinct().collect()

Checking 'price' for reasonable values

In [ ]:
# df.select('price').describe().show()
# q25 = df.approxQuantile(["price"], [0.25], 0.1)[0][0]
# q75 = df.approxQuantile(["price"], [0.75], 0.1)[0][0]
# q90 = df.approxQuantile(["price"], [0.90], 0.1)[0][0]
# q95 = df.approxQuantile(["price"], [0.95], 0.1)[0][0]
# print('25% значение - ', q25)
# print('75% значение - ', q75)
# print('90% значение - ', q90)
# print('95% значение - ', q95)

In [ ]:
# limit upper boundary to 0.9 percentile
df = df.withColumn("price", F.col("price").cast(IntegerType()))
q90 = df.approxQuantile(["price"], [0.90], 0.2)[0][0]
df = df.withColumn("price", F.when(F.col("price") >= q90, q90).otherwise(F.col("price")))
df = df.withColumn("price", F.col("price").cast(IntegerType()))

## Adding new features <a class="anchor" id="feature"></a>



- <b>session duration </b>

  We consider a session - continuous interactions with the markeplace within 30 min. If event_k - event_n > 30, event_k is a start of a new session.

  For each session we count its duration.
- <b>is_first_event_in_session</b>

  Marking each event as a starting (true) or not (false)
- <b>session_id</b>

  Giving each session unique index


- <b>avg_session_duration</b>

  Average session duration per user (sum of unique durations/ sum unique sessions)

- <b>day_time </b>

  Specifying utc_event_time on

  - Early Morning: (06.00.00 - 09.00.00)

  - Morning: (09.00.00 - 12.00.00)

  - Afternoon: (12.00.00 - 17.00.00)

  - Evening: (17.00.00 - 24.00.00)

  - Night (00.00.00 - 06.00.00)

- <b>day_of_week</b>

  Derive a day from the date and write in number

- <b>mean_product_price</b>

  Them product price the user has interected with

- <b>inter_categ</b>

  Number of categories the user paid attention to

- <b>inter_goods</b>

  Number of products the user paid attention to

- <b>avg_goods_per_session </b>

  Average Number of products the user paid attention to throught his sessions

In [ ]:
#time dofference with previous event
my_window = Window.partitionBy("user_id").orderBy("user_id","utc_event_time")
df_new = df.withColumn("id", monotonically_increasing_id()) # add id
df_new = df_new.withColumn("prev_value", F.lag(F.to_timestamp(F.col("utc_event_time"))).over(my_window)) #keep previous records
df_new = df_new.withColumn("min_diff", F.when(F.isnull(F.to_timestamp(F.col("utc_event_time")).cast("long") - F.col("prev_value").cast("long")), np.nan) #count difference in minutes
                              .otherwise(F.from_unixtime((F.to_timestamp(F.col("utc_event_time")).cast("long") - F.to_timestamp(F.col("prev_value")).cast("long")), "HH:mm:ss")))


# df with starts of sessions
sessions_start_df = df_new[(F.col('min_diff') == np.nan) | (F.col('min_diff') > '00:30:00' )]
sessions_start_df = sessions_start_df.withColumn("session_id", F.col('id'))
sessions_start_df = sessions_start_df.select('id','session_id')


#numbering sessions
df_new = df_new.join(sessions_start_df, on=["id"],how='left')

w = Window.orderBy('user_id','utc_event_time')
df_new = df_new.withColumn('session_id', F.when(F.isnull(F.col('session_id')),F.last('session_id', True).over(w)).otherwise(F.col('session_id')))
df_new = df_new.sort(F.col('user_id'),F.col('utc_event_time'))


# start of the session
df_new = df_new.withColumn("is_first_event_in_session", F.col('id') == F.col('session_id'))


# session duration
expr = [F.min(F.col("utc_event_time")),F.max(F.col("utc_event_time"))]
df2 = df_new.groupBy("session_id").agg(*expr)
df2 = df2.withColumn('session_duration',F.from_unixtime(F.to_timestamp(F.col('max(utc_event_time)')).cast('long') - F.to_timestamp(F.col('min(utc_event_time)')).cast('long'),"HH:mm:ss"))
df2 = df2.select('session_id','session_duration')
df_new = df_new.join(df2, on=["session_id"],how='left').drop('id','min_diff','prev_value')


# average session duration per user
df3 = df_new.groupBy("user_id",'session_duration').agg(F.countDistinct('session_duration'))
w = Window.partitionBy("user_id").orderBy('user_id')
df3 = df3.withColumn('avg_session_duration', F.from_unixtime(F.sum(F.to_timestamp(F.col('session_duration')).cast('long')).over(w) / F.sum(F.to_timestamp(F.col('count(session_duration)')).cast('long')).over(w),"HH:mm:ss"))
df3 = df3.select('user_id','avg_session_duration')
df_new = df_new.join(df3, on=["user_id"],how='left')


# adding day_time
df_new = df_new.withColumn('day_time', F.when(F.hour(F.to_timestamp(F.col('utc_event_time'))).between(6, 8),'early morning') \
                          .when(F.hour(F.to_timestamp(F.col('utc_event_time'))).between(9, 11),'morning') \
                          .when(F.hour(F.to_timestamp(F.col('utc_event_time'))).between(12, 16),'afternoon') \
                          .when(F.hour(F.to_timestamp(F.col('utc_event_time'))).between(17, 23),'evening') \
                          .otherwise('night'))


#day_of_week
df_new = df_new.withColumn('day_of_week', F.dayofweek(F.col('utc_event_date')))


#mean_product_price
df3 = df_new.groupBy("user_id").agg(F.mean('price').alias("mean_price"))
df_new = df_new.join(df3, on=["user_id"],how='left')


#inter_categ
df3 = df_new.groupBy("user_id").agg(F.countDistinct('main_category').alias("inter_categ"))
df_new = df_new.join(df3, on=["user_id"],how='left')


#inter_goods
df3 = df_new.groupBy("user_id").agg(F.countDistinct('product_id').alias("inter_goods"))
df_new = df_new.join(df3, on=["user_id"],how='left')


#avg_goods_per_session
df3 = df_new.groupBy("user_id",'session_id').agg(F.countDistinct('product_id').alias("num_of_goods"))
expr = [F.count(F.col("num_of_goods")),F.count(F.col("session_id"))]
df3 = df3.groupBy("user_id").agg(*expr)
df3 = df3.withColumn('avg_goods_per_session', F.col('count(num_of_goods)') / F.col('count(session_id)'))
df_new = df_new.join(df3.select(["user_id",'avg_goods_per_session']), on=["user_id"],how='left')


#changing data types
df_prep = df_new.withColumn("platform", F.col("platform").cast(StringType())) \
    .withColumn('utc_event_time', F.to_timestamp('utc_event_time').cast(DateType())) \
    .withColumn("utc_event_date", F.col("utc_event_date").cast(DateType())) \
    .withColumn("user_id", F.col("user_id").cast(StringType())) \
    .withColumn("event_type", F.col("event_type").cast(StringType())) \
    .withColumn("price", F.col("price").cast(IntegerType())) \
    .withColumn("quantity", F.col("quantity").cast(IntegerType())) \
    .withColumn("product_id", F.col("product_id").cast(StringType())) \
    .withColumn("main_category", F.col("main_category").cast(StringType())) \
    .withColumn("sub_category", F.col("sub_category").cast(StringType())) \
    .withColumn("day_time", F.col("day_time").cast(StringType())) \
    .withColumn("is_first_event_in_session", F.col("is_first_event_in_session").cast(StringType())) \
    .withColumn("session_duration", F.col("session_duration").cast(DateType())) \
    .withColumn("avg_session_duration", F.col("avg_session_duration").cast(DateType())) \
    .withColumn("day_of_week", F.col("day_of_week").cast(IntegerType())) \
    .withColumn("mean_price", F.col("mean_price").cast(IntegerType())) \
    .withColumn("inter_categ", F.col("inter_categ").cast(IntegerType())) \
    .withColumn("inter_goods", F.col("inter_goods").cast(IntegerType())) \
    .withColumn("avg_goods_per_session", F.col("avg_goods_per_session").cast(IntegerType()))

## EDA <a class="anchor" id="EDA"></a>

In [ ]:
#transfer to pandas df
start_time = time.time()
df_new = df_prep.limit(1e5)
pdf = df_new.toPandas()
print("--- %s seconds ---" % (time.time() - start_time))

pdf['utc_event_time'] = pd.to_datetime(pdf['utc_event_time'])
pdf['utc_event_date'] = pd.to_datetime(pdf['utc_event_date'])
pdf['avg_session_duration'] = pd.to_datetime(pdf['avg_session_duration'])
pdf['avg_session_duration'] = pdf['avg_session_duration'].dt.time

for col in ['platform','event_type','day_time']:
  dummies = pd.get_dummies(pdf[col])
  pd.concat([pdf, dummies], axis=1)

--- 1343.5283172130585 seconds ---


1. Basic EDA:
- Columns distribution:
  * `Feature correlation`
  * `Platform Distribution`
  * `Event type Distribution`
  * `Quantity distribution`
  * `Price Distribution`
  * `Number of products in categories`
  * `Number of sub_categories in categories`
  * `Mean and median price in categories`
  * `Day time Distribution`
  * `Day type Distribution`
  * `Number of gadgets per user `
  * `AVG Interacted goods per person`
  * `AVG Interacted goods per session`
  * `Number of prices per product`
  * `First action in session Distribution`
  
- Sessions related insights:
  * `Users per day time Distribution`
  * `Actions per day time Distribution`
  * `Events per session Distribution`
  * `Sessions per user Distribution`
  * `Avg session duration per user Distribution`
  * `Conversion from 'view_item' to 'add_to_cart' ('purchase')`

2. Deep EDA. Hypotheses:
- Is there a correlation between `viewed/bought product price` and a) `time of day` b) `day of week`?
- Is there a correlation between `# of actions` and `'add_to_cart' ('purchase') event`?
- Is there a correlation between `# of sessions` and `'add_to_cart' ('purchase') event`?
- Is there a correlation between `avg session duration` and `'add_to_cart' ('purchase') event`?
- Is there a correlation between `platform type` and  a) `time of day activity` b) `product price`?

### Basic EDA <a class="anchor" id="EDA"></a>

#### Columns distribution <a class="anchor" id="distr"></a>

In [ ]:
corr = pdf.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
df_mask = corr.mask(mask)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy().round(2),
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdBu,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Correlation Heatmap',
    title_x=0.5,
    width=1200,
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_dark'
)

# NaN values are not handled automatically and are displayed in the figure
# So we need to get rid of the text manually
for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.show()

<ipython-input-89-ab5b5f56f209>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]])
fig.add_trace(go.Pie(labels=pdf['platform'].unique(),
                     values=pdf.groupby('platform')['platform'].count(), name="Platform type"), 1, 1)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='Platform Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Platform type', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=pdf['event_type'].unique(),
                     values=pdf.groupby('event_type')['event_type'].count(), name="Event type"))

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='Event_type Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Event type', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = pdf['quantity'].unique(), y = pdf.groupby('quantity')['quantity'].count(),))
fig.update_yaxes(type="log")
fig.update_layout(
    title='Quantity Distribution (log scale)',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    xaxis_title="Quantity",
    yaxis_title="Count",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=pdf[pdf['price']<= 6 * 10**6]['price'], nbinsx=300))
fig.add_vline(x=pdf['price'].mean(), line_width=3, line_dash="dash", line_color="red", label=dict(text="mean",textposition="end"),)
fig.add_vline(x=pdf['price'].median(), line_width=3, line_dash="dash", line_color="yellow",label=dict(text="median", textposition="end"),)
fig.update_yaxes(type="log")
fig.update_layout(
    title= 'Price Distribution',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Price",
    yaxis_title="Count",
)
label=dict(
        text="mean",
        textposition="top right"),
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = pdf['main_category'].unique(), y = pdf.groupby('main_category')['product_id'].count(),))
fig.update_yaxes(type="log")
fig.update_xaxes(visible=True, showticklabels=False)
fig.update_layout(
    title= 'Number of products in categories (log scale)',
    # title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Category",
    yaxis_title="Number of items",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = pdf['main_category'].unique(), y = pdf.groupby('main_category')['sub_category'].nunique(),))
fig.update_xaxes(visible=True, showticklabels=False)
fig.update_layout(
    title= 'Number of sub_categories in categories',
    # title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Category",
    yaxis_title="Number of items",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = pdf['main_category'].unique(), y = pdf.groupby('main_category')['price'].mean(),name='mean'))
fig.add_trace(go.Bar(x = pdf['main_category'].unique(), y = pdf.groupby('main_category')['price'].median(),name='median'))
fig.update_yaxes(type="log")
fig.update_xaxes(visible=True, showticklabels=False)
fig.update_layout(
    title= 'Mean and median price in categories (log scale)',
    # title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Category",
    yaxis_title="Number of items",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=pdf['day_time'].unique(),
                     values=pdf.groupby('day_time')['day_time'].count(), name="Day time"))

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='Day time Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Day time', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=['Понедельник','Вторник','Среда','Четверг','Пятница','Суббота','Воскресенье'],
                     values= pdf.groupby(pdf['utc_event_date'].dt.weekday)['utc_event_date'].count()))

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='Day type Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Day type', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=pdf.groupby('user_id')['platform'].nunique().unique(),
                     values= pdf.groupby('user_id')['platform'].nunique()))

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='Number of gadgets per user Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Number of gadgets per user', x=0.5, y=0.5, font_size=12, showarrow=False)])
fig.show()

In [ ]:
pdf1 = pdf.groupby('user_id')['product_id'].nunique().reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(x = pdf1['product_id'].unique(), y = pdf1.groupby('product_id')['user_id'].nunique()))
fig.update_yaxes(type="log")
fig.update_layout(
    title='AVG Interacted goods per person Distribution (log scale)',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    xaxis_title="Quantity",
    yaxis_title="Count",
)

fig.show()

In [ ]:
#Interacted goods per user
pdf1 = pdf.groupby(['user_id','session_id'])['product_id'].nunique().reset_index()
pdf1 = pdf1.groupby('user_id')[['product_id','session_id']].count()
pdf1['int_goods'] = pdf1['product_id'] / pdf1['session_id']


fig = go.Figure()
fig.add_trace(go.Bar(x = pdf1['int_goods'].unique(), y = pdf1['int_goods'],))
# fig.update_yaxes(type="log")
fig.update_layout(
    title='AVG Interacted goods per session Distribution ',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    xaxis_title="Quantity",
    yaxis_title="Count",
)

fig.show()

In [ ]:
pdf1 = pdf.groupby('product_id')['price'].nunique().sort_values(ascending=False).reset_index()
pdf2 = pd.merge(pdf, pdf1[['product_id','price']], on='product_id', how='left')

fig = go.Figure()
fig.add_trace(go.Bar(x = pdf2['price_y'].unique(), y = pdf2.groupby('price_y')['product_id'].count()))
fig.update_yaxes(type="log")
fig.update_layout(
    title='Number of prices per product Distribution (log scale)',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    xaxis_title="Number of prices",
    yaxis_title="Count",
)

fig.show()

In [ ]:
pdf1 = pdf.groupby('event_type')['is_first_event_in_session'].agg(['count','mean'])
print(pdf1)

fig = go.Figure()
fig.add_trace(go.Pie(labels=pdf['event_type'].unique(),
                     values= pdf.groupby('event_type')['is_first_event_in_session'].agg(['count','mean'])['mean']))

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.7, hoverinfo="label+value+percent")

fig.update_layout(
    title_text='First action in session Distribution',
    title_x = 0.5,
    width=1200,
    height=450,
    template = 'plotly_dark',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='First action', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

             count      mean
event_type                  
add_to_cart   3448  0.968677
purchase       491  0.985743
view_item    48867  0.966787


#### Session-related insights <a class="anchor" id="ses"></a>

In [ ]:
#split into intervals
pdf1 = pdf.groupby(['utc_event_time',pd.Grouper(key = 'utc_event_time', freq='30min')])['user_id'].nunique()
pdf1.index = pdf1.index.set_names(['utc_event_time', 'time_period'])
pdf1 = pdf1.reset_index()
pdf1['only_time_period']= pdf1['time_period'].dt.time
pdf1 = pdf1.drop(['user_id','time_period'],axis=1)
pdf2 = pd.merge(pdf, pdf1, on='utc_event_time', how='left')

fig = go.Figure()
fig.add_trace(go.Bar(x=pdf2['only_time_period'].sort_values().unique(), y = pdf2.groupby('only_time_period')['user_id'].nunique()))
fig.update_layout(
    title= ' Users per Day_time Distribution',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Time",
    yaxis_title="Users",
)

fig.show()

In [ ]:
#split into intervals
pdf1 = pdf.groupby(['utc_event_time',pd.Grouper(key = 'utc_event_time', freq='30min')])[['user_id']].count()
pdf1['count_user_id'] = pdf1['user_id']
pdf1.index = pdf1.index.set_names(['utc_event_time', 'time_period'])
pdf1 = pdf1.reset_index()
pdf1 = pdf1.drop('user_id',axis=1)
pdf1['only_time_period']= pdf1['time_period'].dt.time

fig = go.Figure()
fig.add_trace(go.Bar(x=pdf1['time_period'].dt.time.unique(), y = pdf1.groupby('only_time_period')['count_user_id'].sum()))
fig.update_layout(
    title= 'Actions per Day_time Distribution',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Time",
    yaxis_title="Actions",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=pdf.groupby('session_id')['event_type'].count(),
                           nbinsx=190,))

fig.update_yaxes(type="log")
fig.update_layout(
    title= 'Event per session distribution (log scale)',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Event per session",
    yaxis_title="Count",
)

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=pdf.groupby('user_id')['session_id'].nunique(),))

fig.update_yaxes(type="log")
fig.update_layout(
    title= 'Sessions per user distribution (log scale)',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Sessions per user",
    yaxis_title="Count",
)

fig.show()

In [ ]:
# fig = go.Figure(data=[go.Histogram(x=pdf['avg_session_duration'].sort_values(ascending=False),nbinsx=500)])
# fig.update_layout(title_text= 'Avg session duration per user distribution', xaxis_title="AVG session duration", yaxis_title="Count")
# fig.show()


#split into intervals
pdf1 = pdf.groupby(['avg_session_duration',pd.Grouper(key = 'avg_session_duration', freq='5min')])['user_id'].nunique()
pdf1.index = pdf1.index.set_names(['avg_session_duration', 'avg_session_duration_interval'])
pdf1 = pdf1.reset_index()
pdf1['avg_session_duration_interval']= pdf1['avg_session_duration_interval'].dt.time
pdf1 = pdf1.drop(['user_id'],axis=1)
pdf2 = pd.merge(pdf, pdf1, on='avg_session_duration', how='left')


fig = go.Figure()
fig.add_trace(go.Bar(x=pdf2['avg_session_duration_interval'].sort_values().unique(), y = pdf2.groupby('avg_session_duration_interval')['user_id'].nunique()))
fig.update_yaxes(type="log")
fig.update_layout(
    title= 'Avg session duration per user Distribution (log scale)',
    title_x = 0.5,
    template = 'plotly_dark',
    xaxis_title="Avg session duration",
    yaxis_title="Users",
)

fig.show()

In [ ]:
#Conversion from 'view_item' to 'add_to_cart' ('purchase')

pdf1 = pdf[pdf['event_type'] == 'view_item']
pdf2 = pdf[pdf['event_type'] == 'add_to_cart']
pdf3 = pdf[pdf['event_type'] == 'purchase']
print(f'Conversion:\n"view_item" to "add_to_cart" - {round(len(pdf2) / len(pdf1),3)}\n"add_to_cart" to "purchase" - {round(len(pdf3) / len(pdf2),3)}\n"view_item" to "purchase" - {round(len(pdf3) / len(pdf1),3)}\n ')

Conversion:
"view_item" to "add_to_cart" - 0.071
"add_to_cart" to "purchase" - 0.142
"view_item" to "purchase" - 0.01
 



### Deep EDA <a class="anchor" id="EDA2"></a>

1. Hypothesis: Correlation between `viewed/bought product price` and a) `time of day` b) `day of week`?

  Answer - `No correlation`

In [ ]:
pdf['price_group'] = pd.qcut(pdf['price'],5,duplicates='drop')
pdf['day_of_week'] = pdf['utc_event_date'].dt.day_name()
pdf1 = pdf[(pdf['event_type'] == 'view_item') | (pdf['event_type'] == 'purchase')][['price_group','evening',	'afternoon',	'morning',	'early morning',	'night', 'day_of_week']]
pdf1 = pd.get_dummies(pdf1, columns=['day_of_week'])
pdf1.groupby('price_group').mean().round(2)

,evening,afternoon,morning,early morning,night,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
price_group,,,,,,,,,,,,
"(-0.001, 34000.0]",0.30,0.28,0.17,0.15,0.11,0.13,0.17,0.13,0.13,0.14,0.14,0.15
"(34000.0, 65000.0]",0.29,0.28,0.17,0.15,0.12,0.13,0.17,0.13,0.14,0.14,0.14,0.14
"(65000.0, 119700.0]",0.29,0.28,0.17,0.15,0.12,0.13,0.17,0.13,0.15,0.13,0.15,0.14
"(119700.0, 212500.0]",0.30,0.28,0.16,0.15,0.12,0.13,0.17,0.13,0.14,0.14,0.14,0.15
"(212500.0, 21402000.0]",0.30,0.27,0.16,0.15,0.12,0.13,0.18,0.14,0.15,0.13,0.13,0.14


2. Hypothesis: Correlation between `# of actions` and `'add_to_cart' ('purchase') event`?

  Answer - `No correlation`

In [ ]:
pdf.groupby('user_id').agg({'event_type':['count'], 'add_to_cart' :['mean'], 'purchase':['mean']}).corr()

,,event_type,add_to_cart,purchase
,,count,mean,mean
event_type,count,1.000000,-0.016309,-0.010227
add_to_cart,mean,-0.016309,1.000000,-0.027824
purchase,mean,-0.010227,-0.027824,1.000000


3. Hypothesis: Correlation between `# of sessions` and `'add_to_cart' ('purchase') event`?

  Answer - `No correlation`

In [ ]:
pdf.groupby('user_id').agg({'session_id':['nunique'], 'add_to_cart' :['mean'], 'purchase':['mean']}).corr()

,,session_id,add_to_cart,purchase
,,nunique,mean,mean
session_id,nunique,1.000000,-0.019651,-0.010406
add_to_cart,mean,-0.019651,1.000000,-0.027824
purchase,mean,-0.010406,-0.027824,1.000000


4. Hypothesis: Correlation between `avg session duration` and `'add_to_cart' ('purchase') event`?

  Answer: Yes, the longer `avg session duration`, the less likely the user to `add_to_cart` and `purchase`

In [ ]:
pdf1 = pdf.groupby(['avg_session_duration',pd.Grouper(key = 'avg_session_duration', freq='5min')])['user_id'].nunique()
pdf1.index = pdf1.index.set_names(['avg_session_duration', 'avg_session_duration_interval'])
pdf1 = pdf1.reset_index()
pdf1['avg_session_duration_interval']= pdf1['avg_session_duration_interval'].dt.time
pdf1 = pdf1.drop(['user_id'],axis=1)
pdf2 = pd.merge(pdf, pdf1, on='avg_session_duration', how='left')
pdf2.groupby('avg_session_duration_interval').agg({'add_to_cart' :['mean'], 'purchase':['mean']}).round(2).T

,avg_session_duration_interval,00:00:00,00:05:00,00:10:00,00:15:00,00:20:00,00:25:00,00:30:00,00:35:00,00:40:00,00:50:00
add_to_cart,mean,0.07,0.05,0.05,0.04,0.06,0.07,0.0,0.33,0.0,0.33
purchase,mean,0.01,0.01,0.00,0.01,0.00,0.01,0.0,0.00,0.0,0.00


5. Hypothesis: Correlation between `platform type` and  a) `time of day activity` b) `product price`?

  a)  Answer - `No correlation`

  b)  Answer - `iOS` users tend to interact with more expensive goods. `Android` users show the opposite.

In [ ]:
pdf.groupby('platform')['evening',	'afternoon',	'morning',	'early morning',	'night',].mean().round(2)

<ipython-input-45-2e14cb640867>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,evening,afternoon,morning,early morning,night
platform,,,,,
Android,0.28,0.28,0.17,0.15,0.12
Site,0.26,0.28,0.19,0.16,0.12
iOS,0.32,0.27,0.16,0.14,0.11


In [ ]:
pdf['price_group'] = pd.qcut(pdf['price'],5,duplicates='drop')
pdf.groupby('price_group')['Android', 'Site', 'iOS'].agg(['mean']).round(2)

<ipython-input-46-064cbf6598a8>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Android,Site,iOS
,mean,mean,mean
price_group,,,
"(-0.001, 34000.0]",0.59,0.09,0.32
"(34000.0, 65000.0]",0.60,0.08,0.33
"(65000.0, 119700.0]",0.59,0.08,0.33
"(119700.0, 212500.0]",0.55,0.09,0.36
"(212500.0, 21402000.0]",0.53,0.10,0.37
